In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
#read in reduced csv from files on local machine
#df = pd.read_csv('nashville_reviews.csv')
df = pd.read_csv('/Users/devpatel/Desktop/Yelp Restaurant Recommendation/nashville_reviews.csv')

from datasets import load_dataset

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [4]:
device

device(type='cpu')

In [5]:
df

,Unnamed: 0,text,stars
0,474,Ive been coming to this store since 2008 for a...,5
1,475,Good selection. Friendly staff. Slightly hard ...,5
2,476,Poor poor poor customer service. Id give 0 sta...,1
3,477,I happened to travel across the country and st...,4
4,478,Awful ownership at this smoke shop. I\ve been ...,1
...,...,...,...
452248,6990070,I love it here. Radnor is a short drive from t...,5
452249,6990071,This was such a beautiful park. Me and my fian...,5
452250,6990072,Radnor Lake State Park is has 1400 acres and i...,5
452251,6990073,Love it! The park is beautiful - the lake is ...,5


In [6]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [7]:
df

,text,stars
0,Ive been coming to this store since 2008 for a...,5
1,Good selection. Friendly staff. Slightly hard ...,5
2,Poor poor poor customer service. Id give 0 sta...,1
3,I happened to travel across the country and st...,4
4,Awful ownership at this smoke shop. I\ve been ...,1
...,...,...
452248,I love it here. Radnor is a short drive from t...,5
452249,This was such a beautiful park. Me and my fian...,5
452250,Radnor Lake State Park is has 1400 acres and i...,5
452251,Love it! The park is beautiful - the lake is ...,5


In [8]:
df['stars'].dtype

dtype('int64')

In [9]:
df['stars'] = df['stars'].replace({1: 0, 2: 1, 3: 2, 4: 3, 5: 4})


In [10]:
df

,text,stars
0,Ive been coming to this store since 2008 for a...,4
1,Good selection. Friendly staff. Slightly hard ...,4
2,Poor poor poor customer service. Id give 0 sta...,0
3,I happened to travel across the country and st...,3
4,Awful ownership at this smoke shop. I\ve been ...,0
...,...,...
452248,I love it here. Radnor is a short drive from t...,4
452249,This was such a beautiful park. Me and my fian...,4
452250,Radnor Lake State Park is has 1400 acres and i...,4
452251,Love it! The park is beautiful - the lake is ...,4


In [11]:
type(df)

pandas.core.frame.DataFrame

In [12]:
df = df.rename(columns={'stars': 'labels'})


In [13]:
df

,text,labels
0,Ive been coming to this store since 2008 for a...,4
1,Good selection. Friendly staff. Slightly hard ...,4
2,Poor poor poor customer service. Id give 0 sta...,0
3,I happened to travel across the country and st...,3
4,Awful ownership at this smoke shop. I\ve been ...,0
...,...,...
452248,I love it here. Radnor is a short drive from t...,4
452249,This was such a beautiful park. Me and my fian...,4
452250,Radnor Lake State Park is has 1400 acres and i...,4
452251,Love it! The park is beautiful - the lake is ...,4


In [14]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['labels'], test_size=.2)

In [15]:
train_labels = train_labels.tolist()
val_labels = val_labels.tolist()


In [15]:
type(train_texts)

pandas.core.series.Series

In [16]:
len(train_texts)

361802

In [17]:
len(train_labels)

361802

In [18]:
len(val_texts)

90451

In [16]:
train_texts_list = train_texts.tolist()


In [20]:
type(train_texts_list)

list

In [17]:
val_texts_list = val_texts.tolist()

In [18]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [19]:
train_encodings = tokenizer(train_texts_list, truncation=True, padding=True)


In [ ]:
train_input_ids = train_encodings['input_ids']
len(train_input_ids)

In [ ]:
train_encoding_mask = train_encodings['attention_mask'] # Assuming you're using attention_mask as well
len(train_encoding_mask)

In [20]:
val_encodings = tokenizer(val_texts_list, truncation=True, padding=True)


In [21]:
import torch

class YelpDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = YelpDataset(train_encodings, train_labels)
val_dataset = YelpDataset(val_encodings, val_labels)
#test_dataset = YelpDataset(test_encodings, test_labels)

In [ ]:
len(train_dataset)

In [ ]:
len(val_dataset)

In [ ]:
len(train_encodings)

In [ ]:
train_encodings.items

In [ ]:
type(train_labels)

In [22]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        print("batch complete")

model.eval()


2023-04-06 16:18:44.703139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

KeyboardInterrupt: 

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()